In [100]:
import requests

response = requests.get('https://core.prd.analogio.dk/swagger/v2/swagger.json')
swagger = response.json()

In [101]:
from fuzzingbook.Grammars import Grammar, simple_grammar_fuzzer

SWAG: Grammar = {}

In [111]:
endpoints = []
primitives = set()
for path, reqs in swagger["paths"].items():
	for req_type, details in reqs.items():
		endpoint = f"{req_type.upper()} {path}" 
		if ("parameters" in details and details["parameters"]):
			for param in details["parameters"]:
				name = param["name"]
				param_type = ""
				if "type" in param["schema"]:
					param_type = param["schema"]["type"]
					primitives.add(param_type)
				elif "oneOf" in param["schema"]:
					param_type = param["schema"]["oneOf"][0]["$ref"].split("/")[-1]
				assert not param_type == ""
				assert param["in"] == "path" or param["in"] == "query"
				if param["in"] == "path":
					endpoint = endpoint.replace("{" + name + "}", f"<{param_type}>")
				elif param["in"] == "query" and not "?" in endpoint: # TODO: what if the param is optional
					endpoint = f"{endpoint}?{name}=<{param_type}>"
				elif param["in"] == "query":
					endpoint = f"{endpoint}&{name}=<{param_type}>"
		if ("requestBody" in details and details["requestBody"]):
			model = details["requestBody"]["content"]["application/json"]["schema"]["$ref"].split("/")[-1]
			endpoint += f" requestBody: <{model}>"
		endpoints.append(endpoint)
for modelName, schema in swagger["components"]["schemas"].items():
	if "type" in schema:
		if schema["type"] == "object":
			body = "" 
			for k, param_schema in schema["properties"].items():
				param_type = ""
				if "type" in param_schema:
					param_type = param_schema["type"]
					primitives.add(param_type)
				elif "oneOf" in param_schema:
					param_type = param_schema["oneOf"][0]["$ref"].split("/")[-1]
				assert not param_type == ""
				body += f" \"{k}\":\"<{param_type}>\","
			SWAG[f"<{modelName}>"] = ["{" + body + " }"]
		elif "enum" in schema:
			SWAG[f"<{modelName}>"] = schema["enum"]
		
	# elif "allOf" in schema:  TODO: <<<

In [ ]:
SWAG["<start>"] = ["<endpoint>"]
SWAG["<endpoint>"] = endpoints
# for prim in primitives:
#   print(prim)
SWAG["<string>"] = ["lukas", "swagg", ""]
SWAG["<array>"] = ["[\"i\"]", "[\"item1\", \"item2\"]"] # TODO: we are not checking type of the array!
SWAG["<integer>"] = ["0", "1", "2", "10", "-1"]
SWAG["<boolean>"] = ["true", "false"]


# import json
# print(json.dumps(SWAG, indent=2, sort_keys=True))

for i in range(100):
	print(simple_grammar_fuzzer(SWAG))

POST /api/v2/products requestBody: { "price":"0", "numberOfTickets":"1", "name":"", "description":"lukas", "visible":"false", "allowedUserGroups":"["i"]", "menuItemIds":"["i"]", }
GET /api/v2/menuitems
GET /api/v2/tickets?includeUsed=false
POST /api/v2/vouchers/issue-vouchers requestBody: { "productId":"2", "amount":"2", "voucherPrefix":"lukas", "description":"swagg", "requester":"swagg", }
PUT /api/v2/products/1 requestBody: { "price":"0", "numberOfTickets":"10", "name":"swagg", "description":"", "visible":"false", "allowedUserGroups":"["i"]", "menuItemIds":"["item1", "item2"]", }
PUT /api/v2/account requestBody: { "name":"swagg", "email":"", "privacyActivated":"true", "programmeId":"10", "password":"swagg", }
GET /api/v2/health/check
POST /api/v2/account/auth requestBody: { "token":"", }
GET /api/v2/tickets?includeUsed=true
POST /api/v2/account/resend-verification-email requestBody: { "email":"", }
POST /api/v2/account/resend-verification-email requestBody: { "email":"lukas", }
GET /